In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.stats import SigmaClip, sigma_clipped_stats
from photutils.background import Background2D, MedianBackground
import image_registration

In [2]:
def get_psf(image, psfex_config):
    catfile = image + ".cat"
    psfex_cmd = f"psfex -c {psfex_config} {catfile}"
    print(psfex_cmd)
    try:
        os.system(psfex_cmd)
    except:
        print(f"Could not run PSFex on {image}")

    psf_file_type = ".psf"
    if psf_file_type == ".psf":
        image_psf = image + ".psf"
        psf_data = fits.open(image_psf)[1].data[0][0][0]
    elif psf_file_type == "moffat":
        image_psf = f"{os.path.dirname(image)}/moffat_{os.path.basename(image)}.fits"
        psf_data = fits.open(image_psf)[0].data

    psf_data = psf_data / np.sum(psf_data)
    psf_model_hdu = fits.PrimaryHDU(psf_data)
    psf_model_hdu.writeto(image + ".psfmodel", overwrite=True)

    return psf_data

In [3]:
def psf_fit(backsub_cutout, psf_data):
    backsub_cutout = backsub_cutout[10:50, 10:50]
    psf_data = psf_data[10:50, 10:50]
    xoff, yoff, exoff, eyoff = image_registration.chi2_shifts.chi2_shift(
        psf_data, backsub_cutout, 10, boundary="wrap", nfitted=2, upsample_factor="auto"
    )
    print(f"peak of source detected at offset {xoff, yoff} (in pixel) from target position")

    # Assuming default threshold for forced PSF
    if abs(xoff) > 3.0 or abs(yoff) > 3.0:
        xoff = 0.0
        yoff = 0.0
        phot_type = "Forced_PSF"
    else:
        phot_type = "PSF"

    resize_backsub_cutout = image_registration.fft_tools.shift2d(
        backsub_cutout, -xoff, -yoff
    )
    flux = np.sum(resize_backsub_cutout * psf_data) / np.sum(psf_data * psf_data)
    return flux, phot_type, xoff, yoff

In [4]:

def load_fits(image_path):
    hdu = fits.open(image_path)
    image_data = hdu[0].data
    image_header = hdu[0].header
    return image_data, image_header

In [5]:
def cal_inst_mag(psf_data, image, ra, dec, saturLevel, zero_point):
    image_data, image_header = load_fits(image)
    wcs = WCS(image_header)
    [x_img, y_img] = wcs.all_world2pix(ra, dec, 1)
    x = int(x_img)
    y = int(y_img)
    cutout = Cutout2D(image_data, position=(x, y), size=np.shape(psf_data), wcs=wcs)
    Is = cutout.data
    is_saturated = np.ravel(Is > saturLevel).any()
    sigma_clip = SigmaClip(sigma=3.0)
    try:
        bkg_estimator = MedianBackground()
        bkg = Background2D(
            Is,
            np.shape(psf_data),
            filter_size=(3, 3),
            sigma_clip=sigma_clip,
            bkg_estimator=bkg_estimator,
        )
        background = bkg.background
    except:
        _, background, _ = sigma_clipped_stats(Is)
    sigma_flux = np.sqrt(np.sum(Is * psf_data**2)) / np.sum(psf_data * psf_data)
    Is = Is - background
    flux, PSF_type, xoff, yoff = psf_fit(Is, psf_data)
    source_x = x + xoff
    source_y = y + yoff
    [source_ra, source_dec] = wcs.all_pix2world(source_x, source_y, 1)
    print(f"psf_fit_coord : {source_ra} {source_dec}")
    int_mag = -2.5 * np.log10(flux)
    int_mag_err = 2.5 * np.log10(1 + sigma_flux / flux)
    apparent_mag = int_mag + zero_point  # Apply zero point correction
    return (
        flux,
        sigma_flux,
        apparent_mag,  # Return the apparent magnitude
        int_mag_err,
        PSF_type,
        is_saturated,
        source_ra,
        source_dec,
    )

In [6]:
def get_saturation_level(header):
    # Attempt to retrieve the saturation level from the FITS header or use a default value
    default_saturation_level = 50000  # Replace with a suitable default value
    return header.get('SATURATE', default_saturation_level)

def get_ra_dec(header):
    ra = 210.910674637
    dec = 54.3116510708
    return ra,dec

In [7]:
def process_image(image_path, psfex_config, zero_point):
    psf_data = get_psf(image_path, psfex_config)
    image_data, image_header = load_fits(image_path)
    ra, dec = get_ra_dec(image_header)
    saturLevel = get_saturation_level(image_header)
    results = cal_inst_mag(psf_data, image_path, ra, dec, saturLevel, zero_point)
    return results

In [8]:
def process_all_images(folder_path, psfex_config, zero_point):
    fits_files = [f for f in os.listdir(folder_path) if f.endswith('RA.wcs.proc.fits')]
    all_results = []
    
    for fits_file in fits_files:
        image_path = os.path.join(folder_path, fits_file)
        try:
            result = process_image(image_path, psfex_config, zero_point)
            all_results.append((fits_file, result))
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
    
    return all_results

In [9]:
if __name__ == "__main__":
    folder_path = '/home/aditi/ksp/r-band/data'
    psfex_config = '/home/aditi/ksp/r-band/data/psfex_conf.psfex'
    zero_point = 25.0  # Example zero point value

    all_results = process_all_images(folder_path, psfex_config, zero_point)
    for fits_file, result in all_results:
        print(f"{fits_file}: {result}")

psfex -c /home/aditi/ksp/r-band/data/psfex_conf.psfex /home/aditi/ksp/r-band/data/20230605172226-285-RA.wcs.proc.fits.cat



> WARNING: NTHREADS != 1 ignored: this build of PSFEx is single-threaded


*** PLPLOT WARNING ***
Driver does not support hardware solid fills, switching to software fill.



peak of source detected at offset (-0.666015625, -0.798828125) (in pixel) from target position
psf_fit_coord : 210.91089262352344 54.31192377276919
psfex -c /home/aditi/ksp/r-band/data/psfex_conf.psfex /home/aditi/ksp/r-band/data/20230528151928-738-RA.wcs.proc.fits.cat
Error processing /home/aditi/ksp/r-band/data/20230528151928-738-RA.wcs.proc.fits: [Errno 2] No such file or directory: '/home/aditi/ksp/r-band/data/20230528151928-738-RA.wcs.proc.fits.psf'
psfex -c /home/aditi/ksp/r-band/data/psfex_conf.psfex /home/aditi/ksp/r-band/data/20230521155134-164-RA.wcs.proc.fits.cat
Error processing /home/aditi/ksp/r-band/data/20230521155134-164-RA.wcs.proc.fits: [Errno 2] No such file or directory: '/home/aditi/ksp/r-band/data/20230521155134-164-RA.wcs.proc.fits.psf'
psfex -c /home/aditi/ksp/r-band/data/psfex_conf.psfex /home/aditi/ksp/r-band/data/20230606145220-779-RA.wcs.proc.fits.cat
Error processing /home/aditi/ksp/r-band/data/20230606145220-779-RA.wcs.proc.fits: [Errno 2] No such file or 


> WARNING: NTHREADS != 1 ignored: this build of PSFEx is single-threaded


> *Error*: cannot open /home/aditi/ksp/r-band/data/20230528151928-738-RA.wcs.proc.fits.cat


> WARNING: NTHREADS != 1 ignored: this build of PSFEx is single-threaded


> *Error*: cannot open /home/aditi/ksp/r-band/data/20230521155134-164-RA.wcs.proc.fits.cat


> WARNING: NTHREADS != 1 ignored: this build of PSFEx is single-threaded


> *Error*: cannot open /home/aditi/ksp/r-band/data/20230606145220-779-RA.wcs.proc.fits.cat


> WARNING: NTHREADS != 1 ignored: this build of PSFEx is single-threaded


> *Error*: cannot open /home/aditi/ksp/r-band/data/20230523211044-324-RA.wcs.proc.fits.cat


> WARNING: NTHREADS != 1 ignored: this build of PSFEx is single-threaded


> *Error*: cannot open /home/aditi/ksp/r-band/data/20230526153036-243-RA.wcs.proc.fits.cat


> WARNING: NTHREADS != 1 ignored: this build of PSFEx is single-threaded


> *Error*: cannot open /home/aditi/ksp/r-band/data/20230528151311-870-RA.wcs.proc.fi